In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import matplotlib as mpl
from tqdm import tqdm


from holodeck import plot, detstats
import holodeck.anisotropy as anis
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo
import healpy as hp

import hasasia.sim as hsim
import os


In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 1000
BGL = 1

BUILD_ARRAYS = False
SAVEFIG = True
TOL=0.01
MAXBADS=5

NVARS = 21
# NVARS = 6

NPSRS = 40
NSKIES = 100
# NSKIES = 15

SHOW_GW=True
ALL_TARGETS = ['hard_time', 'gsmf_phi0', 'gsmf_mchar0_log10', 'mmb_mamp_log10', 'mmb_scatter_dex', 'hard_gamma_inner']

LMAX = 8
NSIDE = 8
PARVARS = [0,1,2]

In [ ]:
fobs_cents, _ = holo.utils.pta_freqs()
print(yy.shape)

In [ ]:
jet_map = plot.truncate_colormap('jet_r', 0.1, 0.95
                                 )

In [ ]:
ylabel = "$C_1/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
col0p0 = [
    '#5bb96a', '#5bb96a', 
    '#b98046', '#b98046', 
    '#4680b9', '#4680b9',
]
freq_idx = [0,1,2,3,4]
colors = jet_map(np.linspace(0,1,len(freq_idx)))

labels_fobs=[]
handles=[]
fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False, xscale='linear',
                             height=7)
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    if ii == 2:
        ax.set_ylabel(ylabel)

    xx, yy = detstats.get_anis_var_arrays(targets[ii])
    for ff, fidx in enumerate(freq_idx):
        if ii==0: labels_fobs.append(f"{fobs_cents[fidx]*YR:.2f}")

        for pp in [68,]:
        # for pp in [50]: 
            med, *conf = np.percentile(yy[:,fidx,:], [50, 50-pp/2, 50+pp/2], axis=-1)
            if ff%2 == 0: ax.fill_between(xx, *conf, color=colors[ff], alpha=0.25)
        hh, = ax.plot(xx, med, alpha=0.9, color=colors[ff])
        handles.append(hh)
    
    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    ax.tick_params(axis='x', which='both', top=True, direction='in')    
plt.subplots_adjust(wspace=0.05, hspace=0.35)

fig.legend(handles=handles, labels=labels_fobs, bbox_to_anchor = (0.54, -0.02), loc='lower center',
           ncols = len(freq_idx), title=plot.LABEL_GW_FREQUENCY_YR, title_fontsize=12, 
           columnspacing=1.25)

In [ ]:
ylabel = "$C_1/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
col0p0 = [
    '#5bb96a', '#5bb96a', 
    '#b98046', '#b98046', 
    '#4680b9', '#4680b9',
]
freq_idx = [0,2,4,6,8]
colors = jet_map(np.linspace(0,1,len(freq_idx)))

labels_fobs=[]
handles=[]
fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False, xscale='linear',
                             height=7)
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    if ii == 2:
        ax.set_ylabel(ylabel)

    xx, yy = detstats.get_anis_var_arrays(targets[ii])
    for ff, fidx in enumerate(freq_idx):
        if ii==0: labels_fobs.append(f"{fobs_cents[fidx]*YR:.2f}")

        for pp in [68,]:
        # for pp in [50]: 
            med, *conf = np.percentile(yy[:,fidx,:], [50, 50-pp/2, 50+pp/2], axis=-1)
            if ff%2 == 0: ax.fill_between(xx, *conf, color=colors[ff], alpha=0.25)
        hh, = ax.plot(xx, med, alpha=0.9, color=colors[ff])
        handles.append(hh)
    
    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    ax.tick_params(axis='x', which='both', top=True, direction='in')    
plt.subplots_adjust(wspace=0.05, hspace=0.35)

fig.legend(handles=handles, labels=labels_fobs, bbox_to_anchor = (0.54, -0.02), loc='lower center',
           ncols = len(freq_idx), title=plot.LABEL_GW_FREQUENCY_YR, title_fontsize=12, 
           columnspacing=1.25)

# Anisotropy vs EV

In [ ]:
ff = 1

xlabel = plot.LABEL_EVSS
ylabel = ('$C_1/C_0 (f = %.3f\ \mathrm{yr}^{-1})$' % (fobs_cents[ff]*YR))
colors = cm.rainbow_r(np.linspace(0,1,6))

fig, ax = plot.figax_single(xlabel=xlabel, ylabel=ylabel)

for ii, target in enumerate(targets):
    _, c1c0 = detstats.get_anis_var_arrays(targets[ii]) # V,F,R
    _, evss = detstats.get_ratio_arrays(targets[ii], bgl=BGL) # V,R,S

    xx = np.median(evss, axis=(1,2))
    yy = np.median(c1c0[:,ff,:], axis=-1)

    ax.plot(xx,yy, label=plot.PARAM_KEYS[target], color=colors[ii])
    print(f"{c1c0.shape=}, {evss.shape=}")

ax.legend()

In [ ]:
ff = 1

xlabel = plot.LABEL_EVSS
ylabel = ('$C_1/C_0 (f = %.3f\ \mathrm{yr}^{-1})$' % (fobs_cents[ff]*YR))
colors = cm.rainbow_r(np.linspace(0,1,6))

fig, ax = plot.figax_single(xlabel=xlabel, ylabel=ylabel)

for ii, target in enumerate(targets):
    _, c1c0 = detstats.get_anis_var_arrays(targets[ii]) # V,F,R
    _, evss = detstats.get_ratio_arrays(targets[ii], bgl=BGL) # V,R,S

    xx = np.median(evss, axis=(2))
    yy = c1c0[:,ff,:]
    print(f"{xx.shape=}, {yy.shape=}")

    ax.scatter(xx,yy, label=plot.PARAM_KEYS[target], color=colors[ii], s=1, alpha=0.1)
    print(f"{c1c0.shape=}, {evss.shape=}")

ax.legend()

In [ ]:
nbins = 60
evss_edges = np.logspace(-3, 1.25, nbins)
c1c0_edges = np.logspace(-4, 0, nbins)

target='gsmf_phi0'
_, c1c0 = detstats.get_anis_var_arrays(targets[ii]) # V,F,R
c1c0 = np.repeat(c1c0[:,1,:], NSKIES).reshape(NVARS, NREALS, NSKIES) # V,R,S
_, evss = detstats.get_ratio_arrays(targets[ii], bgl=BGL) # V,R,S

hist, _, _ = np.histogram2d(c1c0.flatten(), evss.flatten(), 
                            bins = (c1c0_edges, evss_edges))
cgrid, egrid = np.meshgrid(c1c0_edges, evss_edges)

In [ ]:
xlabel = plot.LABEL_EVSS
ylabel = ('$C_1/C_0 (f = %.3f\ \mathrm{yr}^{-1})$' % (fobs_cents[ff]*YR))

targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False, 
                             height=7)

fig.text(0.5, 0.08, xlabel, ha='center', va='bottom')
for ii, ax in enumerate(axs.flatten()):
    ax.text(0.02, 0.98, plot.PARAM_KEYS[targets[ii]], transform=ax.transAxes, va='top', ha='left')
    if ii == 2:
        ax.set_ylabel(ylabel)

    _, c1c0 = detstats.get_anis_var_arrays(targets[ii]) # V,F,R
    c1c0 = np.repeat(c1c0[:,1,:], NSKIES).reshape(NVARS, NREALS, NSKIES) # V,R,S
    _, evss = detstats.get_ratio_arrays(targets[ii], bgl=BGL) # V,R,S

    hist, _, _ = np.histogram2d(c1c0.flatten(), evss.flatten(), 
                                bins = (c1c0_edges, evss_edges))
    cgrid, egrid = np.meshgrid(c1c0_edges, evss_edges)
    im = ax.pcolormesh(egrid, cgrid, np.log10(hist))
   
    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    ax.tick_params(axis='x', which='both', top=True, direction='in')    

for ii, ax in enumerate(axs.flatten()):
    if ii<4:
        ax.set_xticklabels([])
plt.subplots_adjust(wspace=0.05, hspace=0.05)


